In [ ]:
# 尝试加载一下数据集，看看数据集格式
# from datasets import load_dataset
# _dataset = load_dataset("fancyzhx/ag_news")
# _dataset

In [1]:
import mteb
from mteb.abstasks.TaskMetadata import TaskMetadata
from mteb.abstasks.AbsTaskClassification import AbsTaskClassification
from typing_extensions import override

class AgNews(AbsTaskClassification):
    metadata = TaskMetadata(
        name="AgNews",
        description="AG is a collection of more than 1 million news articles.",
        dataset={
            "path": "fancyzhx/ag_news",
            "revision": "eb185aade064a813bc0b7f42de02595523103ca4",
        },
        type="Classification",
        category="s2s",
        modalities=["text"],  # text or image
        eval_splits=["test"],
        eval_langs=["eng-Latn"],
        main_score="accuracy",
        date=None,
        domains=None,
        task_subtypes=None,
        license=None,
        annotations_creators=None,
        dialect=None,
        sample_creation=None,
        bibtex_citation="""""",
    )
    # option
    samples_per_label: int = 16

/opt/anaconda3/envs/embed/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/anaconda3/envs/embed/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoModel
_model = AutoModel.from_pretrained("BAAI/bge-small-en-v1.5")

In [27]:
# model = mteb.get_model("BAAI/bge-m3")
raw_model = mteb.get_model("/Users/jie/.cache/modelscope/hub/models/BAAI/bge-small-en-v1___5")

evaluation = mteb.MTEB(tasks=[AgNews(method="kNN-pytorch", n_experiments=8)])
raw_cls_res = evaluation.run(raw_model)

Failed to extract metadata from model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/Users/jie/.cache/modelscope/hub/models/BAAI/bge-small-en-v1___5'. Use `repo_type` argument if needed..
Loader not specified for model /Users/jie/.cache/modelscope/hub/models/BAAI/bge-small-en-v1___5, loading using sentence transformers.


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AgNews, s2s

In [28]:
raw_cls_res[0].scores

{'test': [{'accuracy_cosine': np.float64(0.7784703947368421),
   'f1_cosine': np.float64(0.7754760869125039),
   'accuracy_euclidean': np.float64(0.7784703947368421),
   'f1_euclidean': np.float64(0.7754760869125039),
   'accuracy_dot': np.float64(0.7784703947368421),
   'f1_dot': np.float64(0.7754760869125039),
   'accuracy': np.float64(0.7784703947368421),
   'f1': np.float64(0.7754760869125039),
   'scores_per_experiment': [{'accuracy_cosine': 0.7702631578947369,
     'f1_cosine': 0.7686974948369114,
     'accuracy_euclidean': 0.7702631578947369,
     'f1_euclidean': 0.7686974948369114,
     'accuracy_dot': 0.7702631578947369,
     'f1_dot': 0.7686974948369114,
     'accuracy': 0.7702631578947369,
     'f1': 0.7686974948369114},
    {'accuracy_cosine': 0.7735526315789474,
     'f1_cosine': 0.7682373853553981,
     'accuracy_euclidean': 0.7735526315789474,
     'f1_euclidean': 0.7682373853553981,
     'accuracy_dot': 0.7735526315789474,
     'f1_dot': 0.7682373853553981,
     'accura

In [9]:
import numpy as np
from sentence_transformers import SentenceTransformer

class WrappedModel:
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)

    def encode(self, sentences, **kwargs):
        return self.model.encode(sentences, convert_to_numpy=True, **kwargs)

In [20]:
model = WrappedModel("BAAI/bge-small-en-v1.5")
evaluation = mteb.MTEB(tasks=[AgNews(method="kNN-pytorch", n_experiments=8)])
results = evaluation.run(model)

Failed to extract metadata from model: 'WrappedModel' object has no attribute 'model_card_data'. Upgrading to sentence-transformers v3.0.0 or above is recommended.


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AgNews, s2s

In [21]:
results[0].scores

{'test': [{'accuracy_cosine': 0.77847,
   'f1_cosine': 0.775476,
   'accuracy_euclidean': 0.77847,
   'f1_euclidean': 0.775476,
   'accuracy_dot': 0.77847,
   'f1_dot': 0.775476,
   'accuracy': 0.77847,
   'f1': 0.775476,
   'scores_per_experiment': [{'accuracy_cosine': 0.770263,
     'f1_cosine': 0.768697,
     'accuracy_euclidean': 0.770263,
     'f1_euclidean': 0.768697,
     'accuracy_dot': 0.770263,
     'f1_dot': 0.768697,
     'accuracy': 0.770263,
     'f1': 0.768697},
    {'accuracy_cosine': 0.773553,
     'f1_cosine': 0.768237,
     'accuracy_euclidean': 0.773553,
     'f1_euclidean': 0.768237,
     'accuracy_dot': 0.773553,
     'f1_dot': 0.768237,
     'accuracy': 0.773553,
     'f1': 0.768237},
    {'accuracy_cosine': 0.781316,
     'f1_cosine': 0.780228,
     'accuracy_euclidean': 0.781316,
     'f1_euclidean': 0.780228,
     'accuracy_dot': 0.781316,
     'f1_dot': 0.780228,
     'accuracy': 0.781316,
     'f1': 0.780228},
    {'accuracy_cosine': 0.811579,
     'f1_cosin

## embedding finetune

`fancyzhx/ag_news`： 训练集清洗成目标数据集格式的代码，在 `data_clean.ipynb`中完成

In [ ]:
# 加载微调之后的模型
# ft_model = WrappedModel("output/small/checkpoint-1000")
ft_model = mteb.get_model("output/small/checkpoint-1000")
evaluation = mteb.MTEB(tasks=[AgNews(method="kNN-pytorch", n_experiments=8)])
ft_results = evaluation.run (ft_model)

Failed to extract metadata from model: Repo id must be in the form 'repo_name' or 'namespace/repo_name': 'output/small/checkpoint-1000'. Use `repo_type` argument if needed..
Loader not specified for model output/small/checkpoint-1000, loading using sentence transformers.
No sentence-transformers model found with name output/small/checkpoint-1000. Creating a new one with mean pooling.


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AgNews, s2s

In [30]:
ft_model.model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [26]:
ft_results[0].scores

{'test': [{'accuracy_cosine': np.float64(0.8141611842105263),
   'f1_cosine': np.float64(0.8100538782754227),
   'accuracy_euclidean': np.float64(0.8380592105263157),
   'f1_euclidean': np.float64(0.8348703511303686),
   'accuracy_dot': np.float64(0.7755427631578947),
   'f1_dot': np.float64(0.7692031251355871),
   'accuracy': np.float64(0.8380592105263157),
   'f1': np.float64(0.8348703511303686),
   'scores_per_experiment': [{'accuracy_cosine': 0.8181578947368421,
     'f1_cosine': 0.8125214806026094,
     'accuracy_euclidean': 0.8335526315789473,
     'f1_euclidean': 0.8283917419513277,
     'accuracy_dot': 0.7893421052631578,
     'f1_dot': 0.7822272323057254,
     'accuracy': 0.8335526315789473,
     'f1': 0.8283917419513277},
    {'accuracy_cosine': 0.8196052631578947,
     'f1_cosine': 0.8150724623559217,
     'accuracy_euclidean': 0.8497368421052631,
     'f1_euclidean': 0.8462855047148996,
     'accuracy_dot': 0.7781578947368422,
     'f1_dot': 0.771725244344258,
     'accurac

In [43]:
import os

In [44]:
results = {}
for sub_fold in os.listdir("output"):
    model_dir = os.path.join("output", sub_fold)
    ft_model = mteb.get_model(model_dir)
    evaluation = mteb.MTEB(tasks=[AgNews(method="kNN-pytorch", n_experiments=8)])
    results[sub_fold] = evaluation.run(ft_model)

Failed to extract metadata from model: 401 Client Error. (Request ID: Root=1-6860de03-16741e4259771c8e70877e59;db3d31f1-d9ae-4cd6-9738-e990ea18b637)

Repository Not Found for url: https://huggingface.co/output/same_no_in_batch_neg/resolve/main/README.md.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password..
Loader not specified for model output/same_no_in_batch_neg, loading using sentence transformers.
No sentence-transformers model found with name output/same_no_in_batch_neg. Creating a new one with mean pooling.


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AgNews, s2s

Failed to extract metadata from model: 401 Client Error. (Request ID: Root=1-6860de1c-1a707a7e26c0abdc6eef2513;4b60c406-5ade-4086-b8c2-76c856645584)

Repository Not Found for url: https://huggingface.co/output/no_same_no_in_batch_neg/resolve/main/README.md.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password..
Loader not specified for model output/no_same_no_in_batch_neg, loading using sentence transformers.
No sentence-transformers model found with name output/no_same_no_in_batch_neg. Creating a new one with mean pooling.


...

- AgNews, s2s

Failed to extract metadata from model: 401 Client Error. (Request ID: Root=1-6860de4b-7643628479bd4f0d5fb276bd;5a8b8302-09fa-4270-a49b-c3f21ca72c27)

Repository Not Found for url: https://huggingface.co/output/same/resolve/main/README.md.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password..
Loader not specified for model output/same, loading using sentence transformers.
No sentence-transformers model found with name output/same. Creating a new one with mean pooling.


───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Classification

- AgNews, s2s

In [33]:
len(results)

3

In [ ]:
0.7784703947368421

In [45]:
for k, item in results.items():
    print(k, item[0].scores["test"][0]["accuracy_cosine"])

same_no_in_batch_neg 0.8141611842105263
no_same_no_in_batch_neg 0.7237499999999999
no_same 0.7237499999999999
same 0.8141776315789473
